In [1]:
import os
import numpy
import time
from PIL import Image, ImageDraw

class parameter:
    def __init__(self):
        self.input_nodes=32*64
        self.layer=3
        self.layer_nodes=[256,16,1]
        self.learning_rate=0.01
        self.activation_function_name='leaky_ReLU'
        self.epoch=8
        self.batchsize=4
        self.loss_function_name='cross_entropy'
    def show(self):
        print('input_nodes=',self.input_nodes)
        print('layer=',self.layer)
        print('layer_nodes=',self.layer_nodes)
        print('learning_rate=',self.learning_rate)
        print('activation_function_name=',self.activation_function_name)
        print()
        print('epoch=',self.epoch)
        print('batchsize=',self.batchsize)
        print('loss_function_name=',self.loss_function_name)
        print('')
def activation_function(x,activationfunctionname):
    if activationfunctionname=='sigmoid':
        return 1/(1+numpy.exp(-x))
    if activationfunctionname=='tanh':
        return (numpy.exp(x)-numpy.exp(-x))/(numpy.exp(x)+numpy.exp(-x))
    if activationfunctionname=='ReLU':
        return numpy.maximum(0,x)
    if activationfunctionname=='leaky_ReLU':
        return numpy.maximum(0.01*x,x)

class NeuralNetwork:
    def __init__(self,parameter):
        self.input_nodes=parameter.input_nodes
        self.layer=parameter.layer
        self.layer_nodes=parameter.layer_nodes
        self.w=[numpy.random.normal(0,1,(self.layer_nodes[0],self.input_nodes))]
        self.b=[numpy.array(numpy.random.normal(0,1,(self.layer_nodes[0],1)),ndmin=2)]
        self.mean=numpy.zeros((self.input_nodes,1))
        self.var=numpy.ones((self.input_nodes,1))
        for i in range(1,self.layer):
            self.w.append(numpy.random.normal(0,1,(self.layer_nodes[i],self.layer_nodes[i-1])))
            self.b.append(numpy.array(numpy.random.normal(0,1,(self.layer_nodes[i],1)),ndmin=2))
        self.activation_function_name=parameter.activation_function_name
    def forward_propagation(self,inputlist):
        outputs=numpy.array(inputlist,ndmin=2)
        outputs=(outputs-self.mean)/numpy.sqrt(self.var+0.01)
        for i in range(0,self.layer-1):
            inputs=numpy.dot(self.w[i],outputs)+numpy.array(self.b[i],ndmin=2).reshape(-1,1)
            outputs=activation_function(inputs,self.activation_function_name)
        inputs=numpy.dot(self.w[self.layer-1],outputs)+numpy.array(self.b[self.layer-1],ndmin=2).reshape(-1,1)
        outputs=activation_function(inputs,'sigmoid')
        return outputs

def loadnn(file_name):
    para=parameter()
    f=numpy.loadtxt(file_name+'/parameter')
    para.input_nodes=int(f[0])
    para.layer=int(f[1])
    f=numpy.loadtxt(file_name+'/layer')
    para.layer_nodes=[]
    for i in range(0,para.layer):
        para.layer_nodes.append(int(f[i]))
    f=open(file_name+'/af')
    para.activation_function_name=f.read()
    f.close()
    nn=NeuralNetwork(para)
    for i in range(0,nn.layer):
        nn.w[i]=numpy.array(numpy.loadtxt(file_name+'/w'+str(i)),ndmin=2)
        nn.b[i]=numpy.array(numpy.loadtxt(file_name+'/b'+str(i)),ndmin=2)
    f=numpy.loadtxt(file_name+'/mv')
    nn.mean=f[0,:]
    nn.var=f[1,:]
    nn.mean=numpy.array(nn.mean,ndmin=2).T
    nn.var=numpy.array(nn.var,ndmin=2).T
    return nn


In [2]:
def chooes(file,box_s,nn):
    have_overlap=1
    while(have_overlap==1):
        have_overlap=0
        for i in range(0,len(box_s)):
            if i>=len(box_s):
                break
            for j in range(i+1,len(box_s)):
                if j>=len(box_s):
                    break
                if (box_s[i][0]<box_s[j][2]and box_s[i][2]>box_s[j][0]and box_s[i][1]<box_s[j][3]and box_s[i][3]>box_s[j][1]):
                    have_overlap=1
                    boxa=(max(box_s[i][0],box_s[j][0]),max(box_s[i][1],box_s[j][1]),min(box_s[i][2],box_s[j][2]),min(box_s[i][3],box_s[j][3]))
                    boxo=(min(box_s[i][0],box_s[j][0]),min(box_s[i][1],box_s[j][1]),max(box_s[i][2],box_s[j][2]),max(box_s[i][3],box_s[j][3]))
                    del box_s[j]
                    if(boxa[2]-boxa[0])*(boxa[3]-boxa[1])/((boxo[2]-boxo[0])*(boxo[3]-boxo[1]))>0.3:
                        f=file.crop(boxa)
                        f=f.resize((32,64))
                        f=numpy.array(f)
                        f=f.flatten()
                        f=numpy.array([f,f]).T
                        y=nn.forward_propagation(f)
                        if y[0][0]>0.5:
                            box_s[i]=boxa
                        else:
                            box_s[i]=boxo
    return box_s

In [3]:
nr=loadnn('NNr')
nc=loadnn('NNc')

In [4]:
#标注行人车辆
path='./data/v/'
x_dirs=os.listdir(path)
start_time=time.time()
for x_file in x_dirs:
    fpath=os.path.join(path,x_file)
    file=Image.open(fpath).convert('L')
    [width,hight]=file.size
    box_sr=[]
    box_sc=[]
    test=[]
    tb=[]
    step=[0.1,0.2,0.3]
    for i in step:
        for j in range(0,int((1-i)*width),10):
            for k in range(0,int((1-i)*hight),10):
                box=(j,k,j+i*width,k+i*hight)
                f=file.crop(box)
                f=f.resize((32,64))
                f=numpy.array(f)
                f=f.flatten()
                test.append(f)
                tb.append(box)
    test=numpy.array(test)
    y1=nr.forward_propagation(test.T)
    y2=nc.forward_propagation(test.T)
    for i in range(0,len(tb)):
        if y1[0][i]>0.5:
            box_sr.append(tb[i])
        if y2[0][i]>0.5:
            box_sc.append(tb[i])
    box_sr=chooes(file,box_sr,nr)
    box_sc=chooes(file,box_sc,nc)
    draw = ImageDraw.Draw(file)
    for i in range(0,len(box_sr)):
        draw.line([(box_sr[i][0],box_sr[i][1]),(box_sr[i][0],box_sr[i][3]),(box_sr[i][2],box_sr[i][3]),(box_sr[i][2],box_sr[i][1]),(box_sr[i][0],box_sr[i][1])],fill='black',width=2)
    for i in range(0,len(box_sc)):
        draw.line([(box_sc[i][0],box_sc[i][1]),(box_sc[i][0],box_sc[i][3]),(box_sc[i][2],box_sc[i][3]),(box_sc[i][2],box_sc[i][1]),(box_sc[i][0],box_sc[i][1])],fill='white',width=2)
    file.save('./res/'+x_file)
print(round(time.time()-start_time,2))

506.48


In [5]:
#只标注行人
path='./data/v/'
x_dirs=os.listdir(path)
start_time=time.time()
for x_file in x_dirs:
    fpath=os.path.join(path,x_file)
    file=Image.open(fpath).convert('L')
    [width,hight]=file.size
    box_sr=[]
    test=[]
    tb=[]
    step=[0.1,0.2,0.3]
    for i in step:
        for j in range(0,int((1-i)*width),10):
            for k in range(0,int((1-i)*hight),10):
                box=(j,k,j+i*width,k+i*hight)
                f=file.crop(box)
                f=f.resize((32,64))
                f=numpy.array(f)
                f=f.flatten()
                test.append(f)
                tb.append(box)
    test=numpy.array(test)
    y=nr.forward_propagation(test.T)
    for i in range(0,len(tb)):
                if y[0][i]>0.5:
                    box_sr.append(tb[i])
    box_sr=chooes(file,box_sr,nr)
    draw = ImageDraw.Draw(file)
    for i in range(0,len(box_sr)):
        draw.line([(box_sr[i][0],box_sr[i][1]),(box_sr[i][0],box_sr[i][3]),(box_sr[i][2],box_sr[i][3]),(box_sr[i][2],box_sr[i][1]),(box_sr[i][0],box_sr[i][1])],fill='black',width=2)
    file.save('./resr/'+x_file)
print(round(time.time()-start_time,2))

312.94


In [6]:
#只标注车辆
path='./data/v/'
x_dirs=os.listdir(path)
start_time=time.time()
for x_file in x_dirs:
    fpath=os.path.join(path,x_file)
    file=Image.open(fpath).convert('L')
    [width,hight]=file.size
    box_sc=[]
    test=[]
    tb=[]
    step=[0.1,0.2,0.3]
    for i in step:
        for j in range(0,int((1-i)*width),10):
            for k in range(0,int((1-i)*hight),10):
                box=(j,k,j+i*width,k+i*hight)
                f=file.crop(box)
                f=f.resize((32,64))
                f=numpy.array(f)
                f=f.flatten()
                test.append(f)
                tb.append(box)
    test=numpy.array(test)
    y=nc.forward_propagation(test.T)
    for i in range(0,len(tb)):
                if y[0][i]>0.5:
                    box_sc.append(tb[i])
    box_sc=chooes(file,box_sc,nc)
    draw = ImageDraw.Draw(file)
    for i in range(0,len(box_sc)):
        draw.line([(box_sc[i][0],box_sc[i][1]),(box_sc[i][0],box_sc[i][3]),(box_sc[i][2],box_sc[i][3]),(box_sc[i][2],box_sc[i][1]),(box_sc[i][0],box_sc[i][1])],fill='white',width=2)
    file.save('./resc/'+x_file)
print(round(time.time()-start_time,2))

267.62
